5) NN from the lab - This I tried with the original data and the Preproccess data, and changing a little bit some things like adding layers, batch size and epochs.

In [1]:
# Importing libraries
import keras
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.callbacks import CSVLogger

# importing some libraries
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix


In [2]:
# importing the data
training_data4 = pd.read_csv('data/training_data.csv')
test_data4 = pd.read_csv('data/test_data.csv')

In [3]:
import nltk
# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=5000,tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(training_data4['text'])


C:\Users\Ellis\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(max_features=5000,
                tokenizer=<function word_tokenize at 0x0000029DB39ADC10>)

In [4]:
import keras

# standardize name (X, y) 
X_train = BOW_500.transform(training_data4['text'])
y_train = training_data4['emotion']

X_test = BOW_500.transform(test_data4['text'])


#check dimension
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)

X_train.shape:  (1455563, 5000)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 5000)


In [5]:
# deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
#print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
#y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
#print('y_test.shape: ', y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 0         sadness
1         disgust
2    anticipation
3             joy
Name: emotion, dtype: object

y_train.shape:  (1455563,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]

y_train.shape:  (1455563, 8)


In [6]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  5000
output_shape:  8


In [7]:
# building the model
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500 #training data
X = model_input

# 1st hidden layer
X_W1 = Dense(units=128)(X)  # 64 #hidden layers apply weights to the input
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=128)(H1)  # 64
H2 = ReLU()(H1_W2)          #we can change the functions, maybe gradient descent instead of RelU

# 3rd hidden layer
H2_W3 = Dense(units=128)(H2)  # 64
H3 = ReLU()(H2_W3)          #we can change the functions, maybe gradient descent instead of RelU

# 4rth hidden layer
H3_W4 = Dense(units=128)(H3)  # 64
H4 = ReLU()(H3_W4)          #we can change the functions, maybe gradient descent instead of RelU

# 5th hidden layer
H4_W5 = Dense(units=128)(H4)  # 64
H5 = ReLU()(H4_W5)          #we can change the functions, maybe gradient descent instead of RelU

# output layer
H5_W6 = Dense(units=output_shape)(H5)  # 4 #emotion prediction
H6 = Softmax()(H5_W6)

model_output = H6

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5000)]            0         
                                                                 
 dense (Dense)               (None, 128)               640128    
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 re_lu_2 (ReLU)              (None, 128)               0     

In [34]:
# Trainig the model
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('data/training_log2.csv') #will save our training results into .csv

# training setting
epochs = 10 #total number of iterations in one cicle
batch_size = 128 #number of samples we pass to the network in one pass



# training
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger])
print('training finish')

Epoch 1/10
11372/11372 [==============================] - 228s 20ms/step - loss: 0.8696 - accuracy: 0.6936
Epoch 2/10
11372/11372 [==============================] - 260s 23ms/step - loss: 0.8679 - accuracy: 0.6944
Epoch 3/10
11372/11372 [==============================] - 260s 23ms/step - loss: 0.8661 - accuracy: 0.6951
Epoch 4/10
11372/11372 [==============================] - 272s 24ms/step - loss: 0.8643 - accuracy: 0.6958
Epoch 5/10
11372/11372 [==============================] - 269s 23ms/step - loss: 0.8626 - accuracy: 0.6964
Epoch 6/10
11372/11372 [==============================] - 267s 23ms/step - loss: 0.8611 - accuracy: 0.6969
Epoch 7/10
11372/11372 [==============================] - 269s 23ms/step - loss: 0.8593 - accuracy: 0.6976
Epoch 8/10
11372/11372 [==============================] - 268s 23ms/step - loss: 0.8579 - accuracy: 0.6981
Epoch 9/10
11372/11372 [==============================] - 265s 23ms/step - loss: 0.8562 - accuracy: 0.6988
Epoch 10/10
11372/11372 [============

In [35]:
# predict
pred_result2 = model.predict(X_test, batch_size=128)
pred_result2[:5] #this nombers are in one hot

3219/3219 [==============================] - 16s 5ms/step


array([[9.63178836e-03, 1.06494427e-02, 2.17280891e-02, 1.04156928e-02,
        7.46348619e-01, 8.19519535e-02, 7.97143653e-02, 3.95599231e-02],
       [1.78867107e-04, 1.75515757e-04, 3.31151672e-02, 1.03566209e-02,
        1.62285141e-04, 9.15014744e-01, 3.59923318e-02, 5.00453962e-03],
       [9.48475972e-02, 2.21161135e-02, 3.31467986e-01, 6.32878626e-03,
        2.13750657e-02, 4.79046285e-01, 2.78672352e-02, 1.69509742e-02],
       [2.41411608e-02, 4.66608778e-02, 3.06905657e-01, 1.34962071e-02,
        1.42747849e-01, 1.82823285e-01, 1.82969868e-01, 1.00255124e-01],
       [9.98289045e-03, 4.67738276e-03, 3.60976234e-02, 1.26775410e-02,
        8.45755577e-01, 5.92196770e-02, 4.65327920e-03, 2.69360784e-02]],
      dtype=float32)

In [36]:
# decode to words
pred_result2 = label_decode(label_encoder, pred_result2)
pred_result2[:5] 

array(['joy', 'sadness', 'sadness', 'disgust', 'joy'], dtype=object)

In [37]:
# convert to DF
pred_result2_df=pd.DataFrame(pred_result2)
pred_result2_df.value_counts()

joy             156471
sadness          81986
disgust          56347
anticipation     50961
trust            30831
fear             14328
anger            12765
surprise          8283
dtype: int64

In [40]:
# saving in CSV to later submit to the kaggle
pred_result2_df.to_csv('data/nn_5000_10_1282_5l.csv', index=False, header=True)